# Table round trip for performance testing

This notebook is used to create the data for a "table round-trip", used for performance testing in the `/query` endpoint in the thesis *Performance and price comparison of three popular backend frameworks and the AWS serverless stack*.

The data is created directly in a AWS DynamoDB table with the given name.

The AWS credentials can be set using the `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY` environment variables.

The primary key of the first item is printed at the end of the notebook.

In [4]:
import boto3
from botocore.config import Config
import uuid

In [5]:
# Settings
ROUND_TRIP_LENGTH = 200 # Number of queries that need to be performed to get back to the first primary key
TABLE_NAME = 'round-trip-table'

In [6]:
# Connect to DynamoDB using the boto3 client
botoConfig = Config(
    region_name = 'eu-central-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)
dynamodb = boto3.client('dynamodb', config=botoConfig)

In [7]:
# Check if the table already exists
response = dynamodb.list_tables()
if TABLE_NAME in response['TableNames']:
    print('Table already exists')
else:
    print('Table does not exist')

Table already exists


In [8]:
# Delete the table if it already exists
if TABLE_NAME in response['TableNames']:
    response = dynamodb.delete_table(
        TableName=TABLE_NAME
    )
    print('Table deleted')

Table deleted


In [9]:
# Wait for the table to be deleted (can take a few seconds)
waiter = dynamodb.get_waiter('table_not_exists')
waiter.wait(
    TableName=TABLE_NAME
)

In [10]:
# Create the table
response = dynamodb.create_table(
    TableName=TABLE_NAME,
    KeySchema=[
        {
            'AttributeName': 'primary_key',
            'KeyType': 'HASH'
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'primary_key',
            'AttributeType': 'S'
        }
    ],
    BillingMode='PAY_PER_REQUEST'
)

In [11]:
# Wait for the table to be created (can take a few seconds)
waiter = dynamodb.get_waiter('table_exists')
waiter.wait(
    TableName=TABLE_NAME
)

In [12]:
# Generate initial UUID
initial_primary_key = str(uuid.uuid4())
current_primary_key = initial_primary_key
next_primary_key = str(uuid.uuid4())

# Loop to generate the items
for i in range(ROUND_TRIP_LENGTH - 1):
    response = dynamodb.put_item(
        TableName=TABLE_NAME,
        Item={
            'primary_key': {'S': current_primary_key},
            'next_primary_key': {'S': next_primary_key}
        }
    )
    current_primary_key = next_primary_key # the primary key of the next item is the key we just set as the next primary key
    next_primary_key = str(uuid.uuid4()) # generate a new UUID for the new next primary key
    
# insert last item with the initial primary key as the next primary key to complete the round trip
response = dynamodb.put_item(
    TableName=TABLE_NAME,
    Item={
        'primary_key': {'S': current_primary_key},
        'next_primary_key': {'S': initial_primary_key}
    }
)

In [13]:
# Print the initial primary key
print("The inital primary key is: " + initial_primary_key)

The inital primary key is: 038aca33-a8c0-4b5d-8543-cc50b8f4895c
